In [ ]:
import cv2
import h5py
import imgaug.augmenters as iaa
import numpy as np
import os
from os import getcwd as cwd
from os.path import join as pj
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.utils.data as data
import visdom

# Logger
from IO.logger import Logger
# Data Sampling
from dataset.classification.sampler import adopt_sampling
# model
from model.optimizer import AdamW
from model.resnet.resnet import ResNet
from model.resnet.resnet_base import BasicBlock, Bottleneck, _resnet

In [ ]:
class args:
    experiment_name = "ResNet34_b80_lr1e-4_all02withResize"
    # paths
    all_data_path = pj(cwd(), "data/all_classification_data", "classify_insect_std_20200806_with_size")
    figure_root = pj(cwd(), "figure/image2size", experiment_name)
    model_root = pj(cwd(), "output_model/image2size", experiment_name)
    # train config
    model_name = "resnet34" # choice in ["resnet18", "resnet34"]
    bs = 80
    lr = 1e-4
    nepoch = 300
    weight_func = None  # choice [None, "Pow", "LowSensitivePow"]
    weight_power = 2  # adopt exponent to train loss
    pretrain = True
    param_freeze = False
    sampling = "OverSample" # choice [None, "RandomSample", "OverSample"]
    activation_function = "ReLU" # choice ["ReLU", "LeakyReLU", "RReLU"]
    decoder = None # choice [None, "Concatenate", "FPN"]
    use_dropout = True
    label_smooth = False
    use_aug = True
    use_aug_scale = False
    # visdom
    visdom = True
    port = 8097

In [ ]:
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
if torch.cuda.is_available():
    args.cuda = True
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    args.cuda = False
    torch.set_default_tensor_type('torch.FloatTensor')
torch.multiprocessing.set_start_method('spawn')

#### Save args

In [ ]:
args_logger = Logger(args)
args_logger.save()

### visdom

In [ ]:
if args.visdom:
    # Create visdom
    vis = visdom.Visdom(port=args.port)
    
    win_train_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_test_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='test_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )

In [ ]:
def visualize(vis, phase, visualized_data, window):
    vis.line(
        X=np.array([phase]),
        Y=np.array([visualized_data]),
        update='append',
        win=window
    )

### dataset

In [ ]:
class image2size_dataset(data.Dataset):
    
    def __init__(self, images, sizes=None, label_smooth=False, use_aug=False, use_aug_scale=False, mode="test"):
        # choice mode in ["train", "eval", "test"]
        self.images = images
        self.sizes = sizes
        self.mode = mode
        self.label_smooth = label_smooth
        self.use_aug = use_aug
        self.use_aug_scale = use_aug_scale
        
        if mode == "train":
            print("label_smooth = {}".format(label_smooth))
            print("use_aug = {}".format(use_aug))
            print("use_aug_scale = {}".format(use_aug_scale))
        
        #"""
        aug_list = [
            iaa.OneOf([
                iaa.ShearX((-20, 20)),
                iaa.ShearY((-20, 20))
            ]),
            iaa.OneOf([
                iaa.TranslateX(px=(-20, 20)),
                iaa.TranslateY(px=(-20, 20))
            ]),
            iaa.Rotate((-90, 90)),
            iaa.pillike.Autocontrast(),
            iaa.Invert(0.5),
            iaa.pillike.Equalize(),
            iaa.Solarize(0.5, threshold=(32, 128)),
            iaa.color.Posterize(),
            iaa.pillike.EnhanceContrast(),
            iaa.pillike.EnhanceColor(),
            iaa.pillike.EnhanceBrightness(),
            iaa.pillike.EnhanceSharpness(),
        ]
        #"""
        """
        aug_list = [
            iaa.OneOf([
                iaa.ShearX((-20, 20)),
                iaa.ShearY((-20, 20))
            ]),
            iaa.OneOf([
                iaa.TranslateX(px=(-20, 20)),
                iaa.TranslateY(px=(-20, 20))
            ]),
            iaa.Rotate((-90, 90)),
        ]
        """
        self.aug_seq = iaa.SomeOf((0, 2), aug_list, random_order=True)
        
    def __getitem__(self, index):
        image = self.images[index].astype("uint8")
        if self.mode == "train":
            size = self.sizes[index].astype("float32")
            if self.label_smooth is True:
                size += (np.random.rand() - 0.5) * 5
            if self.use_aug is True:
                image = self.aug_seq(image=image)
                if np.random.rand() < (2/13):
                    ratio = np.random.rand() + 1.0  # 1.0 <= ratio < 2.0
                    image, size = self.aug_scale(image, size, ratio=ratio)
            if self.use_aug_scale is True:
                ratio = np.random.rand() + 1.0  # 1.0 <= ratio < 2.0
                image, size = self.aug_scale(image, size, ratio=ratio)
        
        image = image.astype("float32")
        image = cv2.normalize(image, image, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        image = image.transpose(2,0,1).astype("float32")
        image = torch.from_numpy(image)
        
        if self.mode == "train":
            return image, size
        elif self.mode == "eval":
            size = self.sizes[index].astype("float32")
            return image, size
        else:
            return image
        
    def __len__(self):
        return self.images.shape[0]
    
    def aug_scale(self, image, size, ratio=1.0):
        image = image.astype("float32")
        
        h, w = image.shape[:2]
        src = np.array([[0.0, 0.0],[0.0, 1.0],[1.0, 0.0]], np.float32)
        dest = src * ratio
        h_diff = (ratio - 1) * (h / 2)
        w_diff = (ratio - 1) * (w / 2)
        dest[:, 0] -= w_diff
        dest[:, 1] -= h_diff
        affine = cv2.getAffineTransform(src, dest)
        
        image = cv2.warpAffine(image, affine, (200, 200), cv2.INTER_LANCZOS4)
        size = size * ratio
        return image.astype("uint8"), size

### Loss

In [ ]:
def low_sensitive_pow(loss, power):
    loss_pow = torch.pow(loss, power)
    loss_filter = loss < 1.0
    loss_pow[loss_filter] = loss[loss_filter]
    return loss_pow

### training

In [ ]:
def train(model, train_dataloader, test_dataloader, lr=1e-4, nepoch=100, visdom=False):
    # define loss
    train_l1_loss = nn.L1Loss(reduction='mean')
    test_l1_loss = nn.L1Loss(reduction='mean')
    print("weight_func == {}".format(args.weight_func))
    if args.weight_func == "LowSensitivePow":
        weight_func = low_sensitive_pow
    else:
        weight_func = torch.pow
    
    # define optimizer
    opt = AdamW(model.parameters(), lr=lr)
    # set model train mode
    model.train()
    
    # set best loss
    best_total_test_avg_loss = 1e6
    
    for epoch in range(nepoch):
        total_train_loss = 0
        total_test_loss = 0
        # train
        count = 0
        for image, size in train_dataloader:
            count += 1
            if args.cuda is True:
                image = image.cuda()
                size = size.cuda()
            opt.zero_grad()
            out = model(image)
            train_loss = train_l1_loss(out, size[:, None])
            if args.weight_func == "Pow" or args.weight_func == "LowSensitivePow":
                train_loss = weight_func(train_loss, args.weight_power)
            total_train_loss += train_loss.item()
            train_loss.backward()
            opt.step()
        
        print("train: target_dist = {}, output_dist = {}".format(size[0], out[0].item()))
        total_train_avg_loss = total_train_loss / count
        
        # valid
        model.eval()
        count = 0
        for image, size in test_dataloader:
            count += 1
            if args.cuda is True:
                image = image.cuda()
                size = size.cuda()
            out = model(image)
            test_loss = test_l1_loss(out, size)
            total_test_loss += test_loss.item()
            
        print("test: target_dist = {}, output_dist = {}".format(size[0], out[0].item()))
        total_test_avg_loss = total_test_loss / count
        model.train()
        
        if total_test_avg_loss < best_total_test_avg_loss:
            best_total_test_avg_loss = total_test_avg_loss
            torch.save(model.state_dict(), pj(args.model_root, "valid_" + str(valid_count) + "_best.pth"))
            with open(pj(args.model_root, "valid_" + str(valid_count) + "_best_loss.txt"), mode="w") as f:
                f.write("epoch = {}, test_loss = {}".format(epoch, total_test_avg_loss))
        
        if visdom:
            visualize(vis, epoch+1, total_train_avg_loss, win_train_loss)
            visualize(vis, epoch+1, total_test_avg_loss, win_test_loss)
        print("epoch=%s: train_loss=%f, test_loss=%f" % (epoch, total_train_avg_loss, total_test_avg_loss))
        print("---------------")

### estimation

In [ ]:
def estimate_size(model, dataloader):
    size_array = []
    
    model.eval()
    for image, _ in dataloader:
        if args.cuda is True:
            image = image.cuda()
        out = model(image)
        size_array.extend(out.cpu().detach().numpy())

    model.train()
    return np.array(size_array)

In [ ]:
# load data
with h5py.File(args.all_data_path) as f:
    images = f["X"][:]
    labels = f["Y"][:]
    sizes = f["size"][:]
all_dataset = image2size_dataset(images, sizes, mode="eval")
all_dataloader = data.DataLoader(all_dataset, 1, num_workers=0, shuffle=False)
    
# define kfold
kf = KFold(n_splits=5)
valid_count = 0

# cross validation
total_eval_train_loss = 0
total_eval_test_loss = 0
total_eval_all_loss = 0
for train_index, test_index in kf.split(images):
    print("")
    valid_count += 1
    print("----- valid {} -----".format(valid_count))
    print("")
    # create validation data
    train_index = adopt_sampling(labels, train_index, args.sampling)
    image_train, image_test = images[train_index], images[test_index]
    size_train, size_test = sizes[train_index], sizes[test_index]
    # create dataloader
    train_dataset = image2size_dataset(image_train, size_train, label_smooth=args.label_smooth, use_aug=args.use_aug, use_aug_scale=args.use_aug_scale, mode="train")
    train_dataloader = data.DataLoader(train_dataset, args.bs, num_workers=0, shuffle=True)
    valid_dataset = image2size_dataset(image_train, size_train, mode="eval")
    valid_dataloader = data.DataLoader(valid_dataset, 1, num_workers=0, shuffle=False)
    test_dataset = image2size_dataset(image_test, size_test, mode="eval")
    test_dataloader = data.DataLoader(test_dataset, 1, num_workers=0, shuffle=False)
    
    # create model
    model = ResNet(args.model_name, 1, pretrain=args.pretrain, param_freeze=args.param_freeze, use_dropout=args.use_dropout, activation_function=args.activation_function, decoder=args.decoder).cuda()
    
    # training
    train(model, train_dataloader, test_dataloader, lr=args.lr, nepoch=args.nepoch, visdom=args.visdom)
    
    # evaluation
    model.load_state_dict(torch.load(pj(args.model_root, "valid_" + str(valid_count) + "_best.pth")))
    
    estimated_size_array = estimate_size(model, valid_dataloader)
    eval_train_loss = np.sum(np.abs(estimated_size_array - size_train)) / len(size_train)
    total_eval_train_loss += eval_train_loss
    
    estimated_size_array = estimate_size(model, test_dataloader)
    eval_test_loss = np.sum(np.abs(estimated_size_array - size_test)) / len(size_test)
    total_eval_test_loss += eval_test_loss
    
    estimated_size_array = estimate_size(model, all_dataloader)
    eval_all_loss = np.sum(np.abs(estimated_size_array - sizes)) / len(sizes)
    total_eval_all_loss += eval_all_loss
    
    valid_loss = pd.DataFrame({"train": [eval_train_loss], "test": [eval_test_loss], "all": [eval_all_loss]})
    valid_loss.to_csv(pj(args.figure_root, "valid_loss_" + str(valid_count) + ".csv"))
    
    bbox_df_with_estimate_size = pd.DataFrame({"size": sizes})
    bbox_df_with_estimate_size["eval_size"] = estimated_size_array
    bbox_df_with_estimate_size.to_csv(pj(args.figure_root, "output_size_" + str(valid_count) + ".csv"))

In [ ]:
valid_loss = pd.DataFrame({"train": [total_eval_train_loss / 5], "test": [total_eval_test_loss / 5], "all": [total_eval_all_loss / 5]})
valid_loss.to_csv(pj(args.figure_root, "final_loss_" + str(valid_count) + ".csv"))